In [16]:
from collections import defaultdict, deque
from aocutils.grid import gridneigh

file = 'in.txt'
f = open(file).read().split("\n")
lines = [(line) for line in f]
rows, cols = len(lines), len(lines[0])
grid, neigh = gridneigh(file, to_dict=True, parser=lambda x: list(x))


start = {k for k,v in grid.items() if k[0] == 0 and v == '.'}.pop()
dest = {k for k,v in grid.items() if k[0] == rows-1 and v == '.'}.pop()

tocheck = deque([(start, {start})])
maxx = 0
found = []
while tocheck:
    for _ in range(len(tocheck)):
        loc, seen = tocheck.popleft()
        if loc == dest:
            maxx = max(maxx, len(seen))
            found.append(len(seen)-1)
            continue
        if grid[loc] == '>':
            nei = [(loc[0], loc[1]+1)]
        elif grid[loc] == '<':
            nei = [(loc[0], loc[1]-1)]
        elif grid[loc] == 'v':
            nei = [(loc[0]+1, loc[1])]
        elif grid[loc] == '^':
            nei = [(loc[0]-1, loc[1])]
        else:
            assert grid[loc] == '.'
            nei = neigh[loc]
        for n in nei:
            if n in grid and n not in seen and grid[n] != '#':
                s = seen.copy()
                s.add(n)
                tocheck.append((n,s))       
maxx

2443

In [17]:
def count(nei):
    return sum(grid[tile] != '#' for tile in nei)

splits = {k for k,v in grid.items() if v != '#' and count(neigh[k]) > 2}
splits.add(start)
splits.add(dest)

# calculate distances between splits
splitneigh = defaultdict(dict)
for s in splits:
    tocheck = deque([(s, {s})])
    found = []
    while tocheck:
        for _ in range(len(tocheck)):
            loc, seen = tocheck.popleft()
            nei = neigh[loc]
            for n in nei:
                if n in grid and grid[n] != '#' and n not in seen:
                    if n in splits:
                        splitneigh[start][n] = len(seen)
                    else:
                        s = seen.copy()
                        s.add(n)
                        tocheck.append((n,s))

In [7]:
# check all paths from start to destination with basically same algo as p1, except state is a bit more convoluted
tocheck = deque([(start, {start},0)])
maxx = 0
found = []
while tocheck:
    for _ in range(len(tocheck)):
        loc, seen, dis = tocheck.popleft()
        # print(loc, seen, dis)
        if loc == dest:
            maxx = max(maxx,dis)
            found.append(seen)
            continue
        nei = splitneigh[loc]
        for n,additional in nei.items():
            if n in grid and grid[n] != '#' and n not in seen:
                s = seen.copy()
                s.add(n)
                tocheck.append((n,s, dis + additional))
                
maxx
            

6898